!pip install huggingface-cli
!huggingface-cli login

In [4]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_16_0", "fa", split="train+validation", use_auth_token=True,cache_dir='cv')
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_16_0", "fa", split="test", use_auth_token=True,cache_dir='cv')

print(common_voice)

HF google storage unreachable. Downloading and preparing it from source
Generating train split: 0 examples [00:00, ? examples/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13799it [00:00, 137980.26it/s]
Reading metadata...: 28810it [00:00, 137512.53it/s]
Generating train split: 28810 examples [00:06, 4181.35 examples/s]
Generating validation split: 0 examples [00:00, ? examples/s]
Reading metadata...: 10547it [00:00, 167424.20it/s]
Generating validation split: 10547 examples [00:02, 3918.13 examples/s]
Generating test split: 0 examples [00:00, ? examples/s]
Reading metadata...: 10547it [00:00, 171718.08it/s]
Generating test split: 10547 examples [00:02, 3990.19 examples/s]
Generating other split: 0 examples [00:00, ? examples/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 16309it [00:00, 163083.16it/s]
Reading metadata...: 33390it [00:00, 163295.54it/s]
Generating other split: 33390 examples [00:07, 4198.48 examples/s]
Generating invalidated split: 0 

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 39357
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 10547
    })
})


In [5]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 39357
    })
    test: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 10547
    })
})


In [6]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium",cache_dir='v3')

/home/user/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/utils/hub.py:122: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
preprocessor_config.json: 100%|██████████| 185k/185k [00:00<00:00, 478kB/s]


In [7]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="Persian", task="transcribe",cache_dir='v3')

tokenizer_config.json: 100%|██████████| 805/805 [00:00<00:00, 2.04MB/s]
vocab.json: 100%|██████████| 836k/836k [00:00<00:00, 1.44MB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:01<00:00, 2.40MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 3.81MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 29.9MB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 31.5MB/s]
special_tokens_map.json: 100%|██████████| 2.08k/2.08k [00:00<00:00, 5.66MB/s]


In [8]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="Persian", task="transcribe",cache_dir='v3')

In [9]:
print(common_voice["train"][0])

{'audio': {'path': '/home/user/app/cv/downloads/extracted/3afd7baca3d7c3b86f492c299f17cedab6e37c6d1b71c0ae4ab0e146df03a90e/fa_train_0/common_voice_fa_27232035.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.53045198e-06, -2.47697017e-06, -1.12267401e-06]), 'sampling_rate': 48000}, 'sentence': 'تعداد آنها در حال تقلیل است.', 'variant': ''}


In [10]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [11]:
print(common_voice["train"][0])

{'audio': {'path': '/home/user/app/cv/downloads/extracted/3afd7baca3d7c3b86f492c299f17cedab6e37c6d1b71c0ae4ab0e146df03a90e/fa_train_0/common_voice_fa_27232035.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        5.94547600e-07, -3.05834692e-06, -3.57083627e-06]), 'sampling_rate': 16000}, 'sentence': 'تعداد آنها در حال تقلیل است.', 'variant': ''}


In [12]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [13]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [15]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=10)

Map (num_proc=10): 100%|██████████| 10547/10547 [09:18<00:00, 18.88 examples/s]


In [16]:
common_voice.save_to_disk('common_voice_processed.hf')

Saving the dataset (21/21 shards): 100%|██████████| 10547/10547 [00:09<00:00, 1061.70 examples/s]
